# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
! pip install gmaps

In [7]:
! jupyter nbextension enable --py --sys-prefix gmaps

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# Study data files
weather_metadata = "./Data/weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Carnarvon,-24.87,113.63,68.00,94,90,6.93,AU,1603061442
1,1,Nouadhibou,20.93,-17.03,80.60,47,36,17.22,MR,1603061442
2,2,Severo-Kuril'sk,50.68,156.12,49.59,78,10,11.77,RU,1603061442
3,3,Tasiilaq,65.61,-37.64,33.80,80,74,9.17,GL,1603061442
4,4,East London,-33.02,27.91,62.42,74,32,4.54,ZA,1603061442


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [46]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Lat", "Lng"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity"].astype(float)
maxhumidity = hum_rate.max()

In [47]:
# Create the Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("./Data/Vacation_Heat_Map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,Rosetta,31.40,30.42,73.4,88,0,3.36,EG,1603061435
88,88,Salalah,17.02,54.09,75.2,38,0,3.36,OM,1603061457
157,157,Mīnāb,27.15,57.08,71.6,12,0,6.93,IR,1603061469
299,299,Bandarbeyla,9.49,50.81,78.3,83,0,6.38,SO,1603061492
342,342,Cap Malheureux,-19.98,57.61,73.0,74,0,5.99,MU,1603061499


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['city'] = fifth['city']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Lat']
hotel_df['Lng'] = fifth['Lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,city,Country,Lat,Lng
58,Rosetta,EG,31.40,30.42
88,Salalah,OM,17.02,54.09
157,Mīnāb,IR,27.15,57.08
299,Bandarbeyla,SO,9.49,50.81
342,Cap Malheureux,MU,-19.98,57.61


In [31]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['city']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 58: Rosetta.
Closest hotel to Rosetta at 31.4 , 30.42 is Al Mokhtabar - Moamena Kamel Laboratories.
------------
Retrieving Results for Index 88: Salalah.
Closest hotel to Salalah at 17.02 , 54.09 is Alrehab Palace 1.
------------
Retrieving Results for Index 157: Mīnāb.
Closest hotel to Mīnāb at 27.15 , 57.08 is Central Melli Bank.
------------
Retrieving Results for Index 299: Bandarbeyla.
Closest hotel to Bandarbeyla at 9.49 , 50.81 is REER XOOSHLE GROUP.
------------
Retrieving Results for Index 342: Cap Malheureux.
Closest hotel to Cap Malheureux at -19.98 , 57.61 is Coin de Mire Attitude Hotel.
------------
Retrieving Results for Index 387: Chabahar.
Closest hotel to Chabahar at 25.29 , 60.64 is Melli Bank بانک ملی شعبه بلوار امام.
------------
Retrieving Results for Index 389: Emerald.
Closest hotel to Emerald at -23.53 , 148.17 is EMERALD EXPLORERS INN.
------------
Retrieving Results for Index 420: Caxias do Sul.
Closest hotel to Caxias do Sul at -

In [32]:
hotel_df.head()

,city,Country,Lat,Lng,Hotel Name
58,Rosetta,EG,31.40,30.42,Al Mokhtabar - Moamena Kamel Laboratories
88,Salalah,OM,17.02,54.09,Alrehab Palace 1
157,Mīnāb,IR,27.15,57.08,Central Melli Bank
299,Bandarbeyla,SO,9.49,50.81,REER XOOSHLE GROUP
342,Cap Malheureux,MU,-19.98,57.61,Coin de Mire Attitude Hotel


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Save the image
plt.savefig("./Data/Vacation_Map_With_Hotels.png")

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>